This notebook contains similarity preserving neural network training.

In [1]:
from time import time
from pickle import load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from numpy.random import seed
seed(0)

from keras.models import Model
from keras.layers import Input, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.utils.vis_utils import plot_model
from tensorflow import set_random_seed
set_random_seed(0)

Using TensorFlow backend.


In [2]:
# best so far
params = {'input_shape': [41], 'l1_shape': 96, 'l2_shape': 160, 'l3_shape': 384, 'l4_shape': 512, 'd1_rate': 0.0, 'd2_rate': 0.25, 'distance': 'l1'}


In [3]:
df = pd.read_csv('./dataset.csv', index_col=0)
df.head()

,color,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,...,Music,Romance,Documentary,Musical,Mystery,Horror,History,Animation,Film-Noir,Sport
Avatar,1.0,0.889163,0.529412,0.000000,0.037174,0.001563,1.000000,0.524453,0.007361,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pirates of the Caribbean: At World's End,1.0,0.370690,0.501548,0.024478,0.043478,0.062500,0.406840,0.278865,0.073622,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Spectre,1.0,0.740148,0.436533,0.000000,0.007000,0.017188,0.263080,0.163256,0.017816,0.023256,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Dark Knight Rises,1.0,1.000000,0.486068,0.956522,1.000000,0.042188,0.589253,0.677216,0.162561,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
John Carter,1.0,0.567734,0.386997,0.020652,0.023043,0.001000,0.096066,0.125579,0.002852,0.023256,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# load dict with similar/dissimilar movies
with open('./sims.pkl', 'rb') as f:
    movie_sims = load(f)

In [5]:
def create_pairs(movie_sims, df, split=0.85):
    """Creates positive/negative pairs for one-shot learning"""
    pairs = []
    labels = []

    for movie in movie_sims:
        # get vector for particular movie
        movie_vec = df.loc[movie].values
        # get vectors of its similar/dissimilar movies
        p_vec_l = [df.loc[movie].values for movie in movie_sims[movie]['pos']]
        n_vec_l =[df.loc[movie].values for movie in movie_sims[movie]['neg']]
        # construct pairs
        for pos, neg in zip(p_vec_l, n_vec_l):
            pairs += [[movie_vec, pos]]
            pairs += [[movie_vec, neg]]
            labels += [0, 1]
    
    # validation split
    split_indice = int(split * len(pairs))

    pairs_train = pairs[:split_indice]
    pairs_test = pairs[split_indice:]

    labels_train = labels[:split_indice]
    labels_test = labels[split_indice:]
    
    return ((np.array(pairs_train), np.array(labels_train)),
            (np.array(pairs_test), np.array(labels_test)))

In [6]:
# loss function
def margin_loss(y_true, y_pred):
    m = 1
    loss = 0.5*(1-y_true)*y_pred + 0.5*y_true*K.maximum(0.0, m - y_pred)
    return loss

def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.'''
    pred = y_pred.ravel() > 0.5
    return np.mean(pred == y_true)

def build_model(input_shape,
               l1_shape,
               l2_shape,
               l3_shape,
               l4_shape,
               d1_rate,
               d2_rate,
               distance):
    def build_base_network(input_shape,
                          l1_shape,
                          l2_shape,
                          l3_shape,
                          l4_shape,
                          d1_rate,
                          d2_rate):
        i = Input(shape=input_shape)
        x = Dense(l1_shape, activation='relu')(i)
        if d1_rate:
            x = Dropout(d1_rate)(x)
        x = Dense(l2_shape, activation='relu')(x)
        if d2_rate:
            x = Dropout(d2_rate)(x)
        x = Dense(l3_shape, activation='relu')(x)
        x = Dense(l4_shape, activation='sigmoid')(x)
        return Model(i, x)
    
    base_network = build_base_network(input_shape,
                                     l1_shape,
                                     l2_shape,
                                     l3_shape,
                                     l4_shape,
                                     d1_rate,
                                     d2_rate)

    
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    processed_a = base_network(input_a)
    processed_b = base_network(input_b)

    if distance == 'l1':
        d =  lambda x: K.abs(x[0] - x[1])
    elif distance == 'l2':
        d = lambda x: K.sqrt(K.square(x[0] - x[1]))
    else:
        raise Exception('bad dist')
    
    dist = Lambda(d, 
                  output_shape=lambda x: x[0])([processed_a, processed_b])
    pred = Dense(1, activation='sigmoid')(dist)
    model = Model(input=[input_a, input_b], outputs=[pred])
    model.compile(loss=[margin_loss], optimizer=RMSprop(), metrics=['accuracy'])
    return model, base_network

In [7]:
    print('Creating pairs ...')
    (x_train, y_train), (x_test, y_test) = create_pairs(movie_sims, df)
    print('Pairs created!')
    x_test = x_test.astype('float32')
    
    
    model, base_network = build_model(**params)
    
    plot_model(base_network, to_file='base_network.png', show_shapes=True, show_layer_names=True)
    plot_model(model, to_file='full_model.png', show_shapes=True, show_layer_names=True)
    
    epochs = 30

    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
    history = model.fit([x_train[:, 0], x_train[:, 1]], y_train,
                        batch_size=128,
                        epochs=epochs,
                        validation_data=([x_test[:, 0], x_test[:, 1]], y_test),
                        callbacks=[early_stopping],
                        verbose=0)

Creating pairs ...
Pairs created!


/home/kuba/anaconda3/envs/lsh_tf36/lib/python3.6/site-packages/ipykernel_launcher.py:61: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`


KeyboardInterrupt: 

In [ ]:
y_pred = model.predict([x_test[:, 0], x_test[:, 1]])
y_pred = y_pred.ravel() > 0.5
te_acc = compute_accuracy(y_test, y_pred)

import seaborn as sns
cm = confusion_matrix(y_test, y_pred)
plt.figure(1)
sns.heatmap(cm, annot=True, fmt='d')
plt.title('Confusion matrix')
plt.ylabel('True labels')
plt.xlabel('Model prediction')
plt.savefig('conf_matrix.png')
plt.show()


prec, rec, f1score, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
print(f'Precision: {prec} \nRecall: {rec} \nF1 score: {f1score}')


y_pred = model.predict([x_train[:, 0], x_train[:, 1]])
tr_acc = compute_accuracy(y_train, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

In [ ]:
def plot_history(history, seed=1):
    """Saves plots of traning."""
    plt.figure(seed + 1)
    # Plot training & validation accuracy values
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.savefig('acc_curve.png')
    

    plt.figure(seed + 2)
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.savefig('loss_curve.png')
    
    plt.show()

plot_history(history)

In [ ]:
# compute hash values for each movie
hashes = dict()
for movie in df.index.values:
    movie_v = df.loc[movie].values.reshape(1, 41)
    hash_value = base_network.predict(movie_v)
    # quantization
    hash_value[hash_value < 0.5] = 0
    hash_value[hash_value >= 0.5] = 1
    hashes[movie] = hash_value

In [ ]:
# split hashes for DataFrame
hashes_split = dict()
for movie_name in hashes:
    hashes_split[movie_name] = [int(x) for x in hashes[movie_name][0]]

# load hash values as DataFrame
df_hashes = pd.DataFrame.from_dict(hashes_split, orient='index')
# save them to .csv
df_hashes.to_csv('./neural_hashes.csv', index=True)
df_hashes.head()

In [ ]:
# compute cosine similarity for hashes
cos_h = cosine_similarity(df_hashes.values)
# compute cosine similarity DataFrame for hashes
df_sim_h = pd.DataFrame(cos_h, columns=df_hashes.index.values, index=df_hashes.index.values)
df_sim_h.head()

In [ ]:
# example
df_sim_h['The Matrix'].nlargest(20)
